In [54]:
from scipy.stats import norm
import numpy as np

In [55]:
def npregress(Y, X, bandwidth, Xmidpoints=0):
    
    nrows, ncols = X.shape
    
    YX = np.concatenate((Y, X), axis=1)
    YX = YX[YX[:,1].argsort()]
    
    Y = YX[:, 0]
    X = YX[:, 1]
    
    meanX = np.mean(X)
    stdX = np.std(X)
    iota = np.ones([nrows, 1])
    
    if Xmidpoints == 0:
        nrbins = 100
        
        lowerX = X[int(np.floor(0.01 * nrows + 1))]
        upperX = X[int(np.floor(0.99 * nrows))]
        
        Xmidpoints_used = np.reshape(np.linspace(lowerX, upperX, nrbins), newshape=(nrbins, 1))
        
    else:
        nrbins, ncbins = Xmidpoints.size
        Xmidpoints_used = Xmidpoints
        
    firstb = Xmidpoints_used[0, :]
    lastb = Xmidpoints_used[nrbins - 1, :]
    binsize = ((lastb - firstb) / nrbins)
    
    if bandwidth > 0:
        bandwidth_used = bandwidth
        
    else:
        delta = 0.7764
        bandwidth_used = 1.3643 * delta * stdX * nrows ^ (- 1 / 5)
        
    
    m_regress = np.zeros(nrbins)
    
    for j in range(nrbins):
        Xb = Xmidpoints_used[j, 0]
        Z = (iota * Xb - X) / bandwidth_used
        KX = norm.pdf(Z, loc=0, scale=1)
        
        YKX = Y * X
        m_regress[j] = np.mean(YKX) / np.mean(KX)

        
    return Xmidpoints_used, m_regress, bandwidth_used